In [ ]:
#%load_ext autoreload
#%autoreload 2
#%pdb
#import logging
#logging.basicConfig()
#logging.getLogger().setLevel(logging.DEBUG)
#logging.getLogger().setLevel(logging.INFO)

In [1]:
import os
os.chdir("../")
os.getcwd()

'c:\\Users\\anast\\Documents\\EDU\\Master Thesis\\Experiments\\tda_for_fairness'

In [2]:
# now change into the infembed directory
#%cd infembed
import sys
sys.path.insert(0, 'C:/Users/anast/Documents/EDU/Master Thesis/Experiments/exp/infembed_exp/infembed')

# install the package
! pip install .

Processing c:\users\anast\documents\edu\master thesis\experiments\tda_for_fairness
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for tda-for-fairness: filename=tda_for_fairness-0.1.0-py3-none-any.whl size=1260 sha256=5aeff3794882055d9b86bda59b5de23fb61fe54a43a65e3ae83972536118817d
  Stored in directory: C:\Users\anast\AppData\Local\Temp\pip-ephem-wheel-cache-xnvi338g\wheels\bf\8e\3d\47794c43e1ae3369cb1cf80beb177ef5d02d031793fc4723d5
Successfully built tda-for-fairness
  Attempting uninstall: tda-for-fairness
    Found existing installation: tda-for-fairness 0.1.0
    Uninstalling tda-for-fairness-0.1.0:
      Successfully uninstalled tda-for-fairness-0.1.0


In [3]:
from torch.utils.data import Subset, DataLoader, default_collate, Dataset
import torch.nn as nn
from infembed.clusterer._core.sklearn_clusterer import SklearnClusterer
from infembed.clusterer._core.faiss_clusterer import FAISSClusterer
from infembed.clusterer._core.rule_clusterer import RuleClusterer
from sklearn.cluster import KMeans
from tqdm import tqdm
import pandas as pd
import torch
from infembed.utils.common import Data
import matplotlib.pyplot as plt
from dataclasses import dataclass
import torch.nn.functional as F
import numpy as np
from transformers import DataCollatorWithPadding

c:\Users\anast\AppData\Local\pypoetry\Cache\virtualenvs\tda-for-fairness-5qunVEZW-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### figure out device to compute embeddings on ###

In [4]:
DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print('device:', DEVICE)

device: cpu


#### load model

In [5]:
from transformers import AutoTokenizer, BertForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained('fabriceyhc/bert-base-uncased-ag_news')
_model = BertForSequenceClassification.from_pretrained('fabriceyhc/bert-base-uncased-ag_news')

#### wrap the model

In [6]:
class WrappedModel(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, batch):
        return self.model(**batch)

model = WrappedModel(_model)
model.eval()
model.to(device=DEVICE)

WrappedModel(
  (model): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, b

#### load dataset

In [7]:
from datasets import load_dataset
dataset = load_dataset("ag_news")

#### create the following:
- `eval_dataset`: `Dataset` for evaluation data where the raw text is a field, for use in displaying examples
- `eval_dataloader`: `DataLoader` constructed from `eval_dataset`.  This is given to the `EmbedderBase` implementation's `predict` method.
- `train_dataloader`: `DataLoader` for the training data.  This is given to the `EmbedderBase` implementation's `fit` method.

#### create `eval_dataset`

In [8]:
eval_dataset = dataset['test']
len(eval_dataset)

7600

#### create `eval_dataloader`

first, create a collate function that outputs a tuple, where both elements are what is output by huggingface's `DataCollatorWithPadding`.  This is because `EmbedderBase` implementations expect a batch to be a tuple, where the last element is the label

In [9]:
_collate_fn = DataCollatorWithPadding(tokenizer)

def collate_fn(examples):
    _batch = _collate_fn(examples)
    _batch = {key:val.to(device=DEVICE) for (key, val) in _batch.items()}
    return (_batch, _batch["labels"])

eval_dataset_no_text = dataset["test"]
eval_dataset_no_text = eval_dataset_no_text.map(
    lambda e: tokenizer(e["text"]), batched=True
)
eval_dataset_no_text.set_format(
    type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"]
)
eval_dataloader = torch.utils.data.DataLoader(
    eval_dataset_no_text, collate_fn=collate_fn, batch_size=32
)
batch, labels = next(iter(eval_dataloader))
batch

{'input_ids': tensor([[  101, 10069,  2005,  ...,     0,     0,     0],
         [  101,  1996,  2679,  ...,     0,     0,     0],
         [  101, 18712,  1012,  ...,     0,     0,     0],
         ...,
         [  101, 10478, 19439,  ...,     0,     0,     0],
         [  101,  2027,  1005,  ...,     0,     0,     0],
         [  101,  6505,  4057,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 1, 1, 1, 1, 1, 1])}

pass batch to model to test

In [10]:
model(batch)

SequenceClassifierOutput(loss=tensor(0.1533, grad_fn=<NllLossBackward0>), logits=tensor([[ 1.8282, -3.8458,  2.7387, -2.4806],
        [-1.9454, -4.4805,  0.0072,  5.2491],
        [-1.4580, -4.4780, -0.5301,  5.1840],
        [-1.9375, -4.1853,  0.1949,  4.8149],
        [-0.0570, -4.0112, -1.5208,  4.3649],
        [-2.5590, -4.1836,  1.1783,  4.7290],
        [-2.1090, -4.5176,  0.2823,  5.2207],
        [-2.1285, -4.5300,  0.3136,  5.2297],
        [-1.3974, -4.2017,  0.2179,  4.3100],
        [-1.6491, -3.7142,  0.5749,  3.8798],
        [-2.5590, -4.2059,  1.1764,  4.7442],
        [-2.5594, -4.1536,  1.2848,  4.6271],
        [-2.5203, -4.1828,  1.2189,  4.6663],
        [ 0.0312, -4.1457, -1.6075,  4.4370],
        [-0.0116, -4.1598, -1.5903,  4.4650],
        [-0.0899, -3.8676, -1.4292,  4.2057],
        [-1.7844, -4.5857, -0.1131,  5.2909],
        [-1.4780, -4.2705, -0.5403,  5.0199],
        [-2.1323, -4.3248,  0.3273,  5.0337],
        [-2.3526, -4.1066,  1.1757,  4.4999],

#### create `train_dataloader`

In [12]:
train_dataset_no_text = dataset["train"]
train_dataset_no_text = train_dataset_no_text.shuffle()
print("original training dataset length:", len(train_dataset_no_text))
NUM_TRAIN = 5000
train_dataset_no_text = train_dataset_no_text.select(list(range(NUM_TRAIN)))
train_dataset_no_text = train_dataset_no_text.map(
    lambda e: tokenizer(e["text"]), batched=True
)
train_dataset_no_text.set_format(
    type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "label"]
)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset_no_text, collate_fn=collate_fn, batch_size=32
)
batch, _ = next(iter(train_dataloader))
batch

original training dataset length: 120000


Map: 100%|██████████| 5000/5000 [00:01<00:00, 2837.39 examples/s]


{'input_ids': tensor([[  101,  3748,  1051,  ...,  7659,  1012,   102],
         [  101,  5712,  1001,  ...,     0,     0,     0],
         [  101,  1999,  1996,  ...,     0,     0,     0],
         ...,
         [  101,  2710,  9326,  ...,     0,     0,     0],
         [  101,  2482,  5968,  ...,     0,     0,     0],
         [  101, 22098,  1001,  ...,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'labels': tensor([2, 0, 1, 2, 0, 1, 0, 0, 2, 2, 3, 3, 1, 0, 3, 0, 2, 3, 2, 0, 1, 2, 1, 0,
         2, 0, 2, 1, 3, 1, 0, 3])}

#### define loss function

In [11]:
class HuggingFaceLoss(nn.Module):
    reduction: str = 'sum'
    def forward(self, out, batch):
        return F.cross_entropy(out['logits'], batch['labels'])

loss_fn = HuggingFaceLoss()

#### create embedder

In [13]:
from infembed.embedder._core.arnoldi_embedder import ArnoldiEmbedder
embedder = ArnoldiEmbedder(
    model=model,
    layers=[
        "model.classifier",
        "model.bert.encoder.layer.11",
    ],
    loss_fn=loss_fn,
    sample_wise_grads_per_batch=True,
    arnoldi_dim=200,
    projection_dim=50,
    show_progress=True,
)

C:\Users/anast/Documents/EDU/Master Thesis/Experiments/exp/infembed_exp/infembed\infembed\embedder\_utils\common.py:177: UserWarning: Setting required grads for layer: model.classifier., name: weight
  warnings.warn(
C:\Users/anast/Documents/EDU/Master Thesis/Experiments/exp/infembed_exp/infembed\infembed\embedder\_utils\common.py:177: UserWarning: Setting required grads for layer: model.classifier., name: bias
  warnings.warn(
C:\Users/anast/Documents/EDU/Master Thesis/Experiments/exp/infembed_exp/infembed\infembed\embedder\_utils\common.py:177: UserWarning: Setting required grads for layer: model.bert.encoder.layer.11.attention.self.query, name: weight
  warnings.warn(
C:\Users/anast/Documents/EDU/Master Thesis/Experiments/exp/infembed_exp/infembed\infembed\embedder\_utils\common.py:177: UserWarning: Setting required grads for layer: model.bert.encoder.layer.11.attention.self.query, name: bias
  warnings.warn(
C:\Users/anast/Documents/EDU/Master Thesis/Experiments/exp/infembed_exp/in

consider gradients in the following layers: [Linear(in_features=768, out_features=4, bias=True), Linear(in_features=768, out_features=768, bias=True), Linear(in_features=768, out_features=768, bias=True), Linear(in_features=768, out_features=768, bias=True), Linear(in_features=768, out_features=768, bias=True), Linear(in_features=768, out_features=3072, bias=True), Linear(in_features=3072, out_features=768, bias=True)]


#### fit

In [16]:
embedder.fit(train_dataloader)

INFO:root:start arnoldi iteration
INFO:root:start `_parameter_arnoldi`
Running Arnoldi Iteration for step: 100%|██████████| 200/200 [2:48:48<00:00, 50.64s/it]
INFO:root:start `_parameter_distill`


In [26]:
#with open(f'/content/drive/My Drive/embedder.pkl', 'wb') as f:
#    pickle.dump(embedder, f)

# embedder.save("/content/drive/My Drive/embedder.pkl")
# embedder.load("/content/drive/My Drive/embedder.pkl", projection_on_cpu = True)

In [14]:
import pickle
import io

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

with open(f'C:/Users/anast/Documents/EDU/Master Thesis/Experiments/exp/infembed_exp/embedder.pkl', 'rb') as f:
     embedder = CPU_Unpickler(f).load()

#### compute embeddings

In [15]:
embeddings = embedder.predict(eval_dataloader)

Using ArnoldiEmbedder to compute embeddings. Processing batch:   0%|          | 0/238 [00:00<?, ?it/s]


TypeError: WrappedModel.forward() takes 2 positional arguments but 5 were given

### compute metadata for evaluation data ###
this will be the ingredient needed to display the clusters.  later on, it will also be used by the rule-based clusterer.  therefore, we also add it to the running `Data` instance for easy access.

In [ ]:
def _get_predictions_and_labels(_model, dataloader):
    dfs = []
    for batch in tqdm(dataloader):
        prediction_prob = (
            torch.nn.functional.softmax(_model(*batch[:-1]), dim=1)
            .detach()
            .to(device="cpu")
        )
        prediction_label = torch.argmax(prediction_prob, dim=1).to(device="cpu")
        label = batch[-1].to(
            device="cpu"
        )  # assuming batch is a tensor.  if not, can check
        dfs.append(
            pd.DataFrame(
                {
                    "prediction_label": prediction_label,
                    "label": label,
                    "prediction_prob": list(prediction_prob.numpy()),
                }
            )
        )
    df = pd.concat(dfs, axis=0)
    df.index = list(range(len(df)))
    return df

if True:
    metadata = _get_predictions_and_labels(model, eval_dataloader)

In [ ]:
data.metadata = metadata
data.dataset = eval_dataset

### define clusterer ###

In [ ]:
if False:
    clusterer = SklearnClusterer(sklearn_clusterer=KMeans(n_clusters=25))
if True:
    clusterer = FAISSClusterer(k=25, spherical=True)

### do the clustering ###

In [ ]:
clusters = clusterer.fit_predict(data)

### define ways to display clusters ###
these will all be functions whose input is a list of list of indices in the evaluation dataset

In [ ]:
from infembed.visualization._core.common import (
    DisplayMetadata,
    DisplayPIL,
    DisplayPredictionAndLabels,
    DisplaySingleExamples,
    PerClusterDisplayer,
    DisplayAccuracy,
)

displayers = [
    PerClusterDisplayer(
        [
            DisplayAccuracy(prediction_col="prediction_label", label_col="label"),
            DisplayPredictionAndLabels(
                prediction_col="prediction_label_name", label_col="label_name"
            ),
            # DisplaySingleExamples(
            #     [
            #         DisplayMetadata(["label_name", "prediction_label_name"]),
            #         DisplayPIL(),
            #     ],
            #     limit=3,
            # ),
        ]
    )
]

### display the clusters ###

In [ ]:
for displayer in displayers:
    displayer(clusters, data)

### define rule clusterer ###

In [ ]:
def _accuracy(data):
    return (data.metadata["prediction_label"] == data.metadata["label"]).mean()


def _size(data):
    return len(data)


rule_clusterer = RuleClusterer(
    # clusterer_getter=lambda n_clusters: SklearnClusterer(KMeans(n_clusters=n_clusters)),
    clusterer_getter=lambda n_clusters: FAISSClusterer(k=n_clusters, spherical=True),
    cluster_rule=lambda data: _accuracy(data) < 0.5 and _size(data) >= 10,
    stopping_rule=lambda data: _size(data) < 50,
    max_depth=7,
    branching_factor=3,
)

### do the rule clustering ###

In [ ]:
rule_clusters = rule_clusterer.fit_predict(data)

### display the rule clusters

In [ ]:
for displayer in displayers:
    displayer(rule_clusters, data)

In [ ]:
assert False

In [ ]:
eval_dataset == eval_dataset_no_text

False

In [ ]:
dataset = dataset.map(lambda e: tokenizer(e['text']), batched=True)
dataset

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [ ]:
train_dataset = dataset['train']
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 120000
})

In [ ]:
train_dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [ ]:
collate_fn = DataCollatorWithPadding(tokenizer)

In [ ]:
dataloader = torch.utils.data.DataLoader(train_dataset, collate_fn=collate_fn, batch_size=32)
batch = next(iter(dataloader))
batch

{'input_ids': tensor([[  101,  2813,  2358,  ...,     0,     0,     0],
        [  101, 18431,  2571,  ...,     0,     0,     0],
        [  101,  3514,  1998,  ...,     0,     0,     0],
        ...,
        [  101,  9796, 11014,  ...,     0,     0,     0],
        [  101,  2900,  4517,  ...,     0,     0,     0],
        [  101,  8003, 12235,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])}

In [ ]:
model(**batch)

SequenceClassifierOutput(loss=tensor(0.3592, grad_fn=<NllLossBackward0>), logits=tensor([[-0.8737, -4.0619,  5.4155, -1.7417],
        [-1.1658, -4.0951,  5.4321, -1.4510],
        [-0.8620, -4.1424,  5.5314, -1.7337],
        [ 1.5970, -4.3902,  3.3570, -2.3064],
        [-1.0834, -3.9736,  5.6503, -1.7095],
        [-1.4935, -3.8177,  5.6872, -1.4512],
        [-1.3997, -3.7568,  5.7173, -1.5858],
        [-1.3014, -3.8335,  5.7024, -1.6216],
        [-1.3656, -3.8425,  5.6866, -1.5516],
        [-1.3806, -3.7999,  5.5882, -1.5492],
        [-1.3479, -3.8290,  5.7045, -1.5855],
        [-1.3728, -3.8659,  5.7086, -1.5305],
        [-1.3919, -3.7906,  5.7169, -1.5695],
        [-2.5567, -3.9876,  1.4896,  4.2655],
        [-1.2902, -3.8928,  5.6796, -1.5858],
        [-0.8614, -3.9113,  5.3037, -1.8322],
        [-1.8531, -3.6611,  4.8840, -0.7301],
        [-1.3917, -3.7502,  5.7070, -1.5974],
        [-1.2717, -3.8023,  5.6951, -1.6747],
        [ 0.2828, -4.1816,  4.7058, -2.3063],

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad

In [ ]:
tokenizer.encode([dataset['train'][0]['text'], dataset['train'][1]['text']])

[101,
 2813,
 2358,
 1012,
 6468,
 15020,
 2067,
 2046,
 1996,
 2304,
 1006,
 26665,
 1007,
 26665,
 1011,
 2460,
 1011,
 19041,
 1010,
 2813,
 2395,
 1005,
 1055,
 1040,
 11101,
 2989,
 1032,
 2316,
 1997,
 11087,
 1011,
 22330,
 8713,
 2015,
 1010,
 2024,
 3773,
 2665,
 2153,
 1012,
 102,
 18431,
 2571,
 3504,
 2646,
 3293,
 13395,
 1006,
 26665,
 1007,
 26665,
 1011,
 2797,
 5211,
 3813,
 18431,
 2571,
 2177,
 1010,
 1032,
 2029,
 2038,
 1037,
 5891,
 2005,
 2437,
 2092,
 1011,
 22313,
 1998,
 5681,
 1032,
 6801,
 3248,
 1999,
 1996,
 3639,
 3068,
 1010,
 2038,
 5168,
 2872,
 1032,
 2049,
 29475,
 2006,
 2178,
 2112,
 1997,
 1996,
 3006,
 1012,
 102]

In [ ]:
tokenized = tokenizer([dataset['train'][0]['text'], dataset['train'][1]['text']])

In [ ]:
data_collator([dataset['train'][0], dataset['train'][1]])

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['text', 'label']

> /home/ubuntu/miniconda3/envs/test2/lib/python3.9/site-packages/transformers/tokenization_utils_base.py(3018)pad()
   3016         # The model's main input name, usually `input_ids`, has be passed for padding
   3017         if self.model_input_names[0] not in encoded_inputs:
-> 3018             raise ValueError(
   3019                 "You should supply an encoding or a list of encodings to this method "
   3020                 f"that includes {self.model_input_names[0]}, but you provided {list(encoded_inputs.keys())}"



In [ ]:
d=dataset.map(lambda examples: tokenizer(examples['text']), batched=True)

In [ ]:
d

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [ ]:
from torch.utils.data import DataLoader
next(iter(DataLoader(d['train'], batch_size=3, collate_fn=DataCollatorWithPadding(tokenizer))))

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

> /home/ubuntu/miniconda3/envs/test2/lib/python3.9/site-packages/transformers/tokenization_utils_base.py(752)convert_to_tensors()
    750                         "Please see if a fast version of this tokenizer is available to have this feature available."
    751                     ) from e
--> 752                 raise ValueError(
    753                     "Unable to create tensor, you should probably activate truncation and/or padding with"
    754                     " 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your"



In [ ]:
d=dataset.map(lambda examples: {'g': torch.Tensor([1,2,3])}, batched=True)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

ArrowInvalid: Column 2 named g expected length 1000 but got length 3

> /home/ubuntu/Documents/infembed/infembed/examples/agnews/pyarrow/error.pxi(100)pyarrow.lib.check_status()



In [ ]:
tokenizer(dataset['train'][0])

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

> /home/ubuntu/miniconda3/envs/test2/lib/python3.9/site-packages/transformers/tokenization_utils_base.py(2660)_call_one()
   2658 
   2659         if not _is_valid_text_input(text):
-> 2660             raise ValueError(
   2661                 "text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) "
   2662                 "or `List[List[str]]` (batch of pretokenized examples)."



In [ ]:
dataset['train'][0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2}